# 날씨 데이터셋 val_accuratcy 0.9이상 달성

### <b>데이터 세트 다운로드</b>

* 딥러닝 모델 학습 과정에서 필요한 데이터 세트를 불러온다.

In [ ]:
!git clone https://github.com/ndb796/weather_dataset
%cd weather_dataset

### <b>라이브러리 불러오기(Load Libraries)</b>

* 딥러닝 모델 학습 과정에서 필요한 라이브러리를 불러온다.

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import Image import ImageDataGenerator
from tensorflow.keras import optimizers

import numpy as np
import matplotlib.pyplot as plt

### 하이퍼 파라미터 정의


In [ ]:
# 필요한 하이퍼 파라미터 추가 정의
INPUT_SIZE = 128
BATCH_SIZE = 46
EPOCH = 10
FILTER_SIZE = 3
NUM_FILTERS = 32
MAXPOOL_SIZE = 2

### 데이터 세트 불러오기

In [ ]:
# 폴더 형식으로 구분된 이미지 불러오기
train_generator = ImageDataGenerator(
    rescale = 1/255.,
    validation_split = 0.2,
    horizon_flip = True,
    vertical_flip = True
)
test_generator = ImageDataGenerator(rescale=1/255.)

train_data = train_generator.flow_from_directory(
    directory = 'train/',
    shuffle = True,
    target_size = (INPUT_SIZE, INPUT_SIZE),
    class_mode = 'categorical',
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

val_data = train_generator.flow_from_directory(
    directory = 'train/',
    shuffle = True,
    target_size = (INPUT_SIZE, INPUT_SIZE),
    class_mode = 'categorical',
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

test_data = test_generator.flow_from_directory(
    directory = 'test/',
    shuffle = True,
    target_size = (INPUT_SIZE, INPUT_SIZE),
    class_mode = 'categorical',
    batch_size = BATCH_SIZE
)

### 데이터 시각화

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 60
plt.rcParams.update({'font.size': 20})

class_names = {
  0: "Cloudy",
  1: "Rain",
  2: "Shine",
  3: "Sunrise"
}

images, targets = next(training_set)

print([class_names[x] for x in np.argmax(targets[:4], axis=1)])
plt.imshow(np.concatenate((images[0], images[1], images[2], images[3]), axis=1))
plt.show()

### <b>딥러닝 모델 학습(Training)</b>

* 사전 학습된(pre-trained) 모델(model)을 이용하여 가지고 있는 데이터 세트에 대한 학습이 가능하다.
  * <b>include_top</b>: 네트워크 뒤에 FC (fully-connected) 레이어를 포함하는지 여부
  * <b>weights</b>: None 혹은 "imagenet"
  * 네트워크의 마지막에 FC 레이어를 적용하여 클래스 개수를 일치시킨다.

In [ ]:
# 모델 구성하기
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential([
    Conv2D(NUM_FILTERS, (FILTER_SIZE, FILTER_SIZE), activation='relu', input_shape=(INPUT_SIZE, INPUT_SIZE, 3)),
    MaxPooling2D((MAXPOOL_SIZE, MAXPOOL_SIZE)),
    Dropout(0.2),

    Flatten(),
    Dense(32, activation='relu'),
    Dense(4, activation='softmax')
])

model.summary()

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습(validation_data=validaion_set)
history = model.fit(train_data, epochs=EPOCH, validation_data = val_data)

### 분류 보고서 출력

### <b>학습 결과 확인하기</b>

* 학습 결과를 시각화하여 정상적으로 모델이 학습되었는지 확인한다.

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.title('Accuracy')
plt.legend(['train', 'valid'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.title('Loss')
plt.legend(['train', 'valid'])
plt.show()

In [ ]:
# 학습된 모델 테스트
test_history = model.evaluate(test_set)
test_loss, test_accuracy = test_history

print(f"Test loss: {test_loss:.8f}")
print(f"Test accuracy: {test_accuracy * 100.:.2f}%")